In [ ]:
import torch
import pandas as pd
import numpy as np
import torch.nn as nn
import matplotlib.pyplot as plt
import yfinance as yf
import nbformat
import plotly.express as px
pd.options.plotting.backend = "plotly"

: 

## <u>Model Sharpe</u>

We implement a Sharpe Neural Network aiming to bypass the return prediction, to do so we use 3 different architectures:
-  RNN : Recurrent Neural Network
-  LSTM : Long Short-Term Memory
-  GRU : Gated Recurrent Unit

We use return in percent in order to avoid convergence problems when dealing with values close to zero.


In [1846]:
class NN_Sharpe(nn.Module):

    def __init__(self, input_size, hidden_size, output_size, num_layers, model_name='LSTM'):
        super(NN_Sharpe, self).__init__()

        self.num_layers = num_layers
        self.model_name = model_name

        if self.model_name == 'LSTM':
            self.model = nn.LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, batch_first=True)
        elif self.model_name == 'RNN':
            self.model = nn.RNN(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, batch_first=True)
        else:
            self.model = nn.GRU(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, batch_first=True)
            
        self.linear = nn.Linear(hidden_size, output_size)
    
    def get_alloc(self, x):


        # x : shape[batch_size, seq_length, input_size]

        # output : [batch_size, sequ_length, hidden_size]
        # cn : [num_layers, batch_size, output_size], it's the final state of each layer if proj_size>0 (only for LSTM)
        # hn : [num_layers, batch_size, output_size], it's the final state of each layer

        if self.model_name=="LSTM":
            # if num_layers >1, we only keep the last layer for computing loss

            output, (hn,cn) = self.model(x)
            output, (hn,cn) = output[:,:,:], (hn[:, :, :], cn[:, : , :])
            
        elif self.model_name in ['GRU', 'RNN']:

            output, hn = self.model(x)
            output, hn = output[:,:,:], hn[:, :, :]


        # shape of weights = [batch_size, seq_length, output_size]
        unnormalized_weights = self.linear(output) 
        normalized_weights = torch.softmax(unnormalized_weights, dim=-1)

        return normalized_weights
    
    def get_alloc_last(self, x):

        x = self.get_alloc(x)
        return x[:, -1, :]
    

    def sharpe_loss(self, weights, y):

        # weights : [batch_size, seq_length, output_size]
        # y : [batch_size, seq_length, output_size]

        # returns : [batch_size, seq_length]
        returns = torch.sum(weights*y, dim=-1)

        # mean_returns/std_returns : [batch_size]
        mean_returns = returns.mean(dim=-1)
        std_returns = returns.std(dim=-1) + 1e-12

        mean_sharpe_ratio = mean_returns/std_returns # shape [batch_size]

        return -mean_sharpe_ratio #maximizing the sharpe ratio
    
    def forward(self, x, y):

        x = self.get_alloc(x)
        loss = self.sharpe_loss(x, y).mean()

        return loss



## <u>Getting the data</U>

We import financial data from Yahoo Finance for analysis. The dataset includes daily prices and returns for four diverse assets.

## Assets Overview:
1. **VTI**: Vanguard Total Stock Market Index Fund ETF Shares (U.S. total stock market).
2. **AGG**: iShares Core U.S. Aggregate Bond ETF (U.S. bond market performance).
3. **DBC**: Invesco DB Commodity Index Tracking Fund (commodity exposure, e.g., oil, gold).
4. **^VIX**: CBOE Volatility Index (expected market volatility derived from S&P 500 options).

We retrieve historical daily closing prices for the period from **March 1, 2006**, to **December 31, 2020**.


In [1847]:
tickers = ['VTI', 'AGG', 'DBC', '^VIX']
prices = yf.download(tickers, start="2006-03-01", end="2020-12-31", interval="1d")['Close']
prices.index = prices.index.tz_localize(None).floor('D')
returns = prices.pct_change()*100

returns_prices = pd.concat([prices.shift(1), returns], axis=1, ignore_index=True)
returns_prices.columns = [f'{col}_St-1' for col in prices.columns] + [f'{col}_rt' for col in prices.columns]

[*********************100%***********************]  4 of 4 completed


## Function for Generating Training and Investment Periods

This function is designed to create windows of **training** and **investment** periods based on the specified parameters. The training periods define the data used to train the model, while the investment periods correspond to the intervals where predictions are applied.

### Features:
1. **Customizable Parameters**:
   - `initial_train_years`: Defines the number of years for the initial training period. By default, it is set to **4 years**, as suggested in the *Deep Portfolio Allocation* paper.
   - `retrain_years`: Specifies the number of years between subsequent training and investment periods, defaulting to **2 years**.

2. **Flexible Design**:
   - The function is modular and can handle datasets of varying lengths and frequencies.
   - Automatically adjusts the periods based on the data's index and the defined parameters.

3. **Output**:
   - Returns two lists:
     - **Training periods**: tuples of start and end dates for training.
     - **Investment periods**: tuples of tart and end dates for predictions.

In [1848]:
from datetime import timedelta

def generate_training_periods(data, initial_train_years=4, retrain_years=2):

    # We compute each index for the training & test dates (start and end)

    training_periods = []
    test_periods = []

    last_date_1st_training = data.index[initial_train_years*252]
    training_periods.append((data.index[0], last_date_1st_training))

    first_invest_date = last_date_1st_training
    end_date_first_invest = data.index[data.index.get_loc(first_invest_date) + retrain_years*252]

    test_periods.append((first_invest_date, end_date_first_invest))

    n_periods = (data[data.index >= last_date_1st_training].shape[0]) // (retrain_years*252)

    training_date = first_invest_date

    for i in range(n_periods-1):

        training_date, end_training_date = training_date, data.index[data.index.get_loc(training_date)+retrain_years*252]
        invest_date, end_invest_date = end_training_date, data.index[data.index.get_loc(end_training_date)+retrain_years*252]

        training_periods.append((training_date, end_training_date))
        test_periods.append((invest_date, end_invest_date))

        #print(training_date, end_training_date)
        #print(invest_date, end_invest_date)

        training_date = invest_date


    return training_periods, test_periods




# Function for Computing Rolling Data with Overlap Option

## Overview

The `compute_data` function generates rolling data windows, a common preprocessing step in time series analysis. It is specifically designed to handle both overlapping and non-overlapping windows for training purposes and to extract sequential data for testing.

---

## Features:
1. **Training Mode**:
   - Generates windows of data with both:
     - **Features (\( X \))**: Inputs for the model.
     - **Targets (\( Y \))**: Corresponding outputs.
   - Allows for overlapping or non-overlapping windows depending on the `overlap` parameter.
   
2. **Testing Mode**:
   - Generates sequential windows with only features (\( X \)) for evaluation or prediction.

3. **Flexible Window Configuration**:
   - The `rolling_window` parameter sets the number of time steps included in each window.
   - The function adapts to different use cases by toggling between overlapping and non-overlapping data windows.


In [1849]:
def compute_data(data, start, end, training=True, overlap=True, rolling_window=50):

    rolling_data = []
    idx_start, idx_end = data.index.get_loc(start), data.index.get_loc(end)
    
    #print(len(data))
    if training: 
        data = data.iloc[idx_start:idx_end, :].copy()

        if overlap:

            for i in range(len(data), rolling_window+1, -1):
                    
                #print(i-rolling_window, i)
                rolling_data.append((data.iloc[i - rolling_window - 1: i-1, :], data.iloc[i - rolling_window: i, :]))

            return rolling_data[::-1]
        
        else:

            for i in range(len(data), rolling_window+1, -rolling_window):
                    
                #print(i-rolling_window, i)
                rolling_data.append((data.iloc[i - rolling_window - 1: i-1, :], data.iloc[i - rolling_window: i, :]))

            return rolling_data[::-1]
    

    else:
        
        for i in range(idx_start, idx_end):
                
            rolling_data.append((data.iloc[i-rolling_window: i, :]))

        return rolling_data


In [1850]:
dates = generate_training_periods(returns)
start, end = dates[0][0]
data= compute_data(data=returns, start=start, end=end, overlap=False, training=True)

In [1851]:
print(start, end)
print(len(data))
print(data[0])

2006-03-01 00:00:00 2010-03-03 00:00:00
20
(Ticker           AGG       DBC       VTI       ^VIX
Date                                               
2006-03-10  0.040356 -0.435731  0.781865  -6.545741
2006-03-13  0.181517  1.312907  0.242926  -4.050637
2006-03-14  0.332194  1.468683  1.031901  -5.540898
2006-03-15 -0.270891 -1.106855  0.448779   5.679708
2006-03-16  0.422533  0.860959  0.215682   5.550653
2006-03-17 -0.220398 -0.469486  0.076857   1.168617
2006-03-20 -0.040162 -2.058317 -0.614442  -2.722772
2006-03-21 -0.261144  0.350262 -0.556415  -1.441901
2006-03-22  0.181269 -0.174524  0.606166  -3.528398
2006-03-23 -0.080420  1.704551 -0.023184  -0.356824
2006-03-24  0.171024 -0.214874  0.131356   0.179047
2006-03-27  0.080347  0.602934 -0.069453   2.412873
2006-03-28 -0.250878  1.284243 -0.586823   1.047119
2006-03-29 -0.201212  0.676246  0.932036  -5.440416
2006-03-30 -0.453626  1.217468 -0.269327   5.662100
2006-03-31  0.334179 -0.705102  0.054006  -1.555742
2006-04-03 -0.484460

In [1852]:
from torch.utils.data import DataLoader, TensorDataset


def training(data_used, input_size,
              hidden_size, output_size, num_layers, model_name, 
              initial_train_years=4, retrain_years=2, rolling_window=50, 
              shuffle=False, batch_size=64, epoch=50, overlap=True):
    
    # We make a copy of the datafram
    result = data_used.copy()
    for col in result.columns:
        alloc_col = f"{col}_alloc"
        result[alloc_col] = np.nan

    model = NN_Sharpe(input_size, hidden_size, output_size, num_layers, model_name)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
    scheduler_global = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.8)
    periods_train, periods_invest = generate_training_periods(data_used, initial_train_years, retrain_years)

    
    for i in range(len(periods_train)):

        start_training, end_training = periods_train[i][0], periods_train[i][1]
        start_invest, end_invest = periods_invest[i][0], periods_invest[i][1]

        print(f'training from {start_training} to {end_training}')
        print(f'invest from {start_invest} to {end_invest}')

        data_training = compute_data(data=returns, start=start_training, end=end_training, rolling_window=rolling_window, training=True, overlap=overlap)
        data_invest = compute_data(data=returns, start=start_invest, end=end_invest, training=False, rolling_window=rolling_window)

        X_tensor = torch.tensor([df[0].values for df in data_training], dtype=torch.float32)
        print(X_tensor.shape)
        Y_tensor = torch.tensor([df[1].values for df in data_training], dtype=torch.float32)
        X_test = torch.tensor([df.values for df in data_invest], dtype=torch.float32)
        dataset = TensorDataset(X_tensor, Y_tensor)

        dataloader = DataLoader(dataset, batch_size, shuffle)
        global_loss = []

        for epoch in range(epoch): 

            loss = []
        
            for k, (batch_x, batch_y) in enumerate(dataloader):
                optimizer.zero_grad()
                outputs = model(batch_x, batch_y)
                loss.append(outputs.item())
                outputs.backward()
                optimizer.step()
            
            loss_epoch = sum(loss) / len(loss)
            print(f'epoch {epoch}, loss = {loss_epoch}')
            global_loss.append(global_loss)


        with torch.no_grad():
            alloc_test = model.get_alloc_last(X_test)

        scheduler_global.step()

        filtered_data = data_used[(data_used.index >= start_invest) & (data_used.index < end_invest)]

        #print("Filtered data index shape:", len(filtered_data.index))
        #print("Alloc test shape:", alloc_test.numpy().shape)
        #print("Alloc columns:", [f"{col}_alloc" for col in result.columns if not col.endswith("_alloc")])


        alloc_columns = [f"{col}_alloc" for col in result.columns if not col.endswith("_alloc")]
        returns_columns = [col for col in result.columns if not col.endswith("_alloc")]

        result.loc[filtered_data.index, alloc_columns] = alloc_test.numpy()

        #t.append(data_training)
        #j.append(data_invest)

    rt_pf = 0
    for i in range(len(returns_columns)):
        rt_pf += result[alloc_columns[i]] * result[returns_columns[i]]
    result['return_pf'] = rt_pf


    return result



        #for data in data_training:

        #    data_X = torch.tensor(data[0].values, dtype=torch.float32).view(1, rolling_window, input_size)
        #    data_Y = torch.tensor(data[1].values, dtype=torch.float32).view(1, rolling_window, input_size)
        #    print(model(data_X, data_Y))

            



        




        

In [1853]:
import plotly.express as px

def get_result(dataframe_result, prices):

    fig_prices = px.line(prices[prices.index.isin(dataframe_result.dropna().index)], 
                        title="prices")
    fig_prices.show()

    alloc_columns = [col for col in dataframe_result.columns if col.endswith('_alloc')]
    fig_allocations = px.line(dataframe_result.dropna(), y=alloc_columns, 
                            title="allocations")
    fig_allocations.show()

    N = dataframe_result.dropna().shape[0]
    log_returns_sum = np.sum(np.log(1+dataframe_result['return_pf']/100))
    annual_rt = np.exp((252/N) * log_returns_sum) - 1
    sharpe = dataframe_result['return_pf'].mean() / dataframe_result['return_pf'].std() * np.sqrt(252)

    print(f"nb de jours d'investissement: {N}\nannualized return: {annual_rt}")
    print(f"sharpe ratio: {sharpe}")
    print(f"std deviation: {(dataframe_result['return_pf']/100).std() * np.sqrt(252)}")
    print(f"downside_risk:  { ((dataframe_result[ dataframe_result['return_pf']<0 ]['return_pf']) / 100).std() * np.sqrt(252)} ")



In [1854]:

result = training(
    data_used=returns.dropna(), 
    input_size=4, 
    hidden_size=64, 
    output_size=4, 
    num_layers=1, 
    model_name='GRU', 
    initial_train_years=4, 
    retrain_years=2, 
    rolling_window=10, 
    shuffle=True, 
    epoch=100,
    batch_size=2**6,
    overlap=False
)

training from 2006-03-02 00:00:00 to 2010-03-04 00:00:00
invest from 2010-03-04 00:00:00 to 2012-03-02 00:00:00
torch.Size([100, 10, 4])
epoch 0, loss = 0.012882491806522012
epoch 1, loss = 0.020321122836321592
epoch 2, loss = 0.011899519013240933
epoch 3, loss = 0.019914539996534586
epoch 4, loss = 0.008912450168281794
epoch 5, loss = 0.023411341942846775
epoch 6, loss = 0.01001064432784915
epoch 7, loss = 0.011670438572764397
epoch 8, loss = 0.011325484607368708
epoch 9, loss = 0.012209109729155898
epoch 10, loss = 0.00477686885278672
epoch 11, loss = 0.008811954874545336
epoch 12, loss = -0.004039238207042217
epoch 13, loss = -0.009578602388501167
epoch 14, loss = -0.010591775178909302
epoch 15, loss = -0.0043048192746937275
epoch 16, loss = -0.003121868707239628
epoch 17, loss = -0.012941626831889153
epoch 18, loss = -0.0002163611352443695
epoch 19, loss = -0.014514061622321606
epoch 20, loss = -0.011804121546447277
epoch 21, loss = -0.0069090500473976135
epoch 22, loss = -0.007377

In [1855]:
get_result(result, prices)

nb de jours d'investissement: 2520
annualized return: 0.3991907515492372
sharpe ratio: 2.024159894895029
std deviation: 0.17333452402344587
downside_risk:  0.09141073633294869 


In [1856]:
(result.dropna()['return_pf']/100).mean()*252

np.float64(0.35085679192897795)